# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
!pip install langchain_community langchain_experimental langchain_openai

In [2]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [3]:
%pip install faiss-cpu > /dev/null
%pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [5]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
SERPAPI_API_KEY = os.environ["SERPAPI_API_KEY"]


In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.schema import Document
import faiss

embedding_model = OpenAIEmbeddings()

embedding_size = 1536

index = faiss.IndexFlatL2(embedding_size)

docstore = InMemoryDocstore({})
index_to_docstore_id = {}

vectorstore = FAISS(
    embedding_model.embed_query,
    index,
    docstore,
    index_to_docstore_id,
)


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [8]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

study_plan_prompt = PromptTemplate.from_template(
    "Create a study plan for a high school student with this goal: {objective}"
)

llm = OpenAI(temperature=0, max_tokens=500)
study_plan_chain = LLMChain(llm=llm, prompt=study_plan_prompt)

tools = [
    Tool(
        name="StudyPlanner",
        func=study_plan_chain.run,
        description="Generates study plans. Input: academic goal. Output: a short study plan.",
    ),
]

prefix = """You are an AI who creates study plans. Objective: {objective}."""
suffix = """Task: {task} {agent_scratchpad}"""


prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task"],
)




<ipython-input-8-281ea35f7d77>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  study_plan_chain = LLMChain(llm=llm, prompt=study_plan_prompt)


In [9]:
llm = OpenAI(temperature=0, max_tokens=100)

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
llm = OpenAI(
    temperature=0,
    max_tokens=200,
    model_name="gpt-3.5-turbo-16k"
)


<ipython-input-9-3024d2840633>:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [10]:
OBJECTIVE = "Help student prepare for science exam"



In [11]:
verbose = False

max_iterations: Optional[int] = 3

baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)


In [13]:
from langchain.chat_models import ChatOpenAI


In [14]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")


<ipython-input-14-0a233f0c1333>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")


In [18]:
objective = "Help a high school student improve in math and prepare for university entrance exams."
agent_executor.run({"objective": objective, "task": "Create a personalized study plan"})




> Entering new AgentExecutor chain...
Action: StudyPlanner
Action Input: High school student looking to improve in math and prepare for university entrance exams
Observation: Study Plan for Improving Math Skills and Preparing for University Entrance Exams:

1. Assess Current Math Skills:
- Take a diagnostic test to identify strengths and weaknesses in math topics.
- Review past math assignments and tests to pinpoint areas that need improvement.

2. Set Specific Goals:
- Determine a target score for university entrance exams.
- Break down goals into smaller, manageable tasks to work on each week.

3. Create a Study Schedule:
- Allocate dedicated time each day for math practice and review.
- Include a mix of problem-solving, concept review, and practice tests in the schedule.

4. Utilize Resources:
- Use textbooks, online resources, and math apps to supplement learning.
- Seek help from teachers, tutors, or online forums for clarification on difficult topics.

5. Practice Regularly:
- 

'Study Plan for Improving Math Skills and Preparing for University Entrance Exams:\n\n1. Assess Current Math Skills:\n- Take a diagnostic test to identify strengths and weaknesses in math topics.\n- Review past math assignments and tests to pinpoint areas that need improvement.\n\n2. Set Specific Goals:\n- Determine a target score for university entrance exams.\n- Break down goals into smaller, manageable tasks to work on each week.\n\n3. Create a Study Schedule:\n- Allocate dedicated time each day for math practice and review.\n- Include a mix of problem-solving, concept review, and practice tests in the schedule.\n\n4. Utilize Resources:\n- Use textbooks, online resources, and math apps to supplement learning.\n- Seek help from teachers, tutors, or online forums for clarification on difficult topics.\n\n5. Practice Regularly:\n- Work on math problems daily to build problem-solving skills.\n- Review previous topics regularly to reinforce understanding.\n\n6. Take Practice Tests:\n- Ta

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

study_plan_prompt = PromptTemplate.from_template(
    "You are an academic advisor. Create a study plan for the following objective: {objective}"
)
study_plan_chain = LLMChain(llm=llm, prompt=study_plan_prompt)

tools = [
    Tool(
        name="StudyPlanner",
        func=study_plan_chain.run,
        description="Creates study plans based on academic objectives.",
    )
]

prefix = "You are a helpful AI. Objective: {objective}"
suffix = """Task: {task}
{agent_scratchpad}

Respond in the following format:
Action: <tool name>
Action Input: <input>
"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["objective", "task"]
)

llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)
